This notebook is to extract fish from image.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import os
import time
import glob
import cv2
import multiprocessing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
from sklearn import cluster
import matplotlib.pyplot as plt
import tensorflow as tf


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input/train"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
def load_train():
    train_img = []
    input_file_path = "../input/train/"
    fishes = ['ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT']
    print("loading files starts at " + time.asctime())
    for idx in range(len(fishes)):
        files = glob.glob(input_file_path + fishes[idx] + "/*")
        print(fishes[idx])
        for file in files:
            img = cv2.imread(file)
            img = cv2.resize(img, (500,500), cv2.INTER_LINEAR)
            train_img.append(img)
        print(fishes[idx], len(train_img))
    print("finish loading files at " + time.asctime())
    return train_img
train = load_train()

In [ ]:
#tensorflow to calculate


In [ ]:
def get_random_data(train):
    np_train = np.array(train)
    index = np.random.choice(np.arange(np_train.shape[0]), size = [100])
    index.sort()
    np_train = np_train[index]
    return np_train

small = get_random_data(train)




In [ ]:
def dist(args):
    a, b = args
    a = (a - a.mean()) / a.std()
    b = (b - b.mean()) / b.std()
    return np.mean(np.abs(a - b))

pool = multiprocessing.Pool(8)

dis_mat = np.zeros([small.shape[0], small.shape[0]])
print("start " + time.asctime())
for idx, img in enumerate(small):
    imgs = [(img[0], f[0]) for f in small]
    dists = pool.map(dist, imgs)
    print(idx)
    dis_mat[idx, :] = dists

print(dis_mat[:10])



In [ ]:
cls = cluster.DBSCAN(metric='precomputed', min_samples=5, eps=0.7)
y = cls.fit_predict(dis_mat)
print(y)
print(pd.Series(y).value_counts())

res = []
for idx in range(np.min(y), np.max(y) + 1):
    tmp = small[y == idx]
    res.append(tmp)
res = np.array(res)

print(res[0].shape)

In [ ]:
plt.figure(figsize = [20, 5])
plt.subplot(141)
plt.imshow(res[0][0][0])
plt.subplot(142)
plt.imshow(res[0][1][0])
plt.subplot(143)
plt.imshow(res[0][6][0])
plt.subplot(144)
plt.imshow(res[0][7][0])
plt.show()
   